In [13]:
from openai import OpenAI
client = OpenAI(api_key='INPUT API KEY')

In [14]:
file = client.files.create(
file=open("Intake_Form.pdf", "rb"),
purpose='assistants'
)

In [3]:
assistant = client.beta.assistants.create(
instructions = "Role and Context: You are an AI assistant for an attorney specializing in real estate property tax. You have access to a standard intake form called Intake_Form.pdf. Task Overview: When a user interacts with you, 1. identify and list all the required fields (blank spaces) in the Intake_Form.pdf; 2.Ask the user to provide the information needed for those fields; 3.If the user cannot provide all the required details, fill in only the fields for which information was provided; 4.Leave blank any fields for which no information was provided, ensuring the overall layout and formatting remain consistent with the original form; 5.Generate a PDF (partial or complete) that includes: All supplied information in the correct fields and Any remaining blank fields clearly left empty (with the same label/formatting); 6.Generate user_data.json to store the provided information with the name of the field as keys and the provided information as values. Important Requirements: 1.Always preserve the original form's format, labels, and structure; 2.If certain fields are not filled out, those fields must remain visibly blank in the final document; 3.You must never remove or alter any existing text, labels, or structure in the form; 4.The final PDF output should contain all provided information in the correct fields, even if some fields remain empty; 5. Remember to generate user_data.json to store already provided information.",
model="gpt-4o",
tools=[{"type": "code_interpreter"}],
tool_resources={
  "code_interpreter": {
    "file_ids": [file.id]
  }
}
)

In [4]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_1mJIUQZmBEAvn0O337QECrPZ', created_at=1738272947, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [5]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content="I need an attorney in property tax. Can you help me?"
)

In [6]:
run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=assistant.id
)

In [7]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)


user: I need an attorney in property tax. Can you help me?
assistant: Yes, I can assist you with property tax-related inquiries. To proceed, let's start by reviewing the Intake_Form.pdf you've uploaded. I'll identify the required fields for you to fill out, and then you can provide the necessary information. 

Let me first examine the uploaded document to list all the fields we need to fill.
assistant: The extracted text from the PDF shows several fields that you need to fill out. Here are some of the key fields identified:

1. County
2. Township
3. PIN(s) (Property Identification Number)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase? (Yes/No)
7. Demolition? (Yes/No)
8. Demo Permit?
9. Paid Receipt?
10. Demo Affidavit?
11. Excess Land?
12. Non-Buildable Area (Conservancy, Wetlands, Detention Pond)?
13. New Building Completed When?
14. Have Occupancy Permit?
15. Vacancy 2024
16. Vacancy 2025
17. Is property being offered for sale? (Yes/No)
18. Physical problems wi

In [8]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content="1. Cook; 2. Northfield. The rest is still unknown. Could you generate the form filled with these two blanks fielded?"
)

In [9]:
run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=assistant.id
)

In [10]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)




user: I need an attorney in property tax. Can you help me?
assistant: Yes, I can assist you with property tax-related inquiries. To proceed, let's start by reviewing the Intake_Form.pdf you've uploaded. I'll identify the required fields for you to fill out, and then you can provide the necessary information. 

Let me first examine the uploaded document to list all the fields we need to fill.
assistant: The extracted text from the PDF shows several fields that you need to fill out. Here are some of the key fields identified:

1. County
2. Township
3. PIN(s) (Property Identification Number)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase? (Yes/No)
7. Demolition? (Yes/No)
8. Demo Permit?
9. Paid Receipt?
10. Demo Affidavit?
11. Excess Land?
12. Non-Buildable Area (Conservancy, Wetlands, Detention Pond)?
13. New Building Completed When?
14. Have Occupancy Permit?
15. Vacancy 2024
16. Vacancy 2025
17. Is property being offered for sale? (Yes/No)
18. Physical problems wi

In [11]:
print(messages.data[0])

Message(id='msg_aMblTMU3sqZxmU7QS6zRZqBh', assistant_id='asst_3lWgtpIxV3x04whvMI97yTaC', attachments=[Attachment(file_id='file-RMcmqX4dnP7MbXjyfzZxMR', tools=[CodeInterpreterTool(type='code_interpreter')]), Attachment(file_id='file-6uEdyWPHuZ3uW7bbpXj7dD', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=179, file_path=FilePath(file_id='file-6uEdyWPHuZ3uW7bbpXj7dD'), start_index=146, text='sandbox:/mnt/data/filled_form.pdf', type='file_path'), FilePathAnnotation(end_index=310, file_path=FilePath(file_id='file-RMcmqX4dnP7MbXjyfzZxMR'), start_index=278, text='sandbox:/mnt/data/user_data.json', type='file_path')], value="I've generated the filled PDF form with the information you provided. You can download the file using the link below:\n\n[Download the filled form](sandbox:/mnt/data/filled_form.pdf)\n\nI've also created a JSON file to store the information you provided:\n\n[Download us

In [12]:
doc_names = ['user_data.json', 'filled_intake_form.pdf']
for idx in range(len(messages.data[0].attachments)):
        
    file_data = client.files.content(messages.data[0].attachments[idx].file_id)
    file_data_bytes = file_data.read()

    with open(doc_names[idx], "wb") as f:
            f.write(file_data_bytes)